**Data Exploration**

One of the most difficult thing or the problem in data analysis is most of the time we dont know our data. Even though every thing is in front of us.Still we remain far from the reality. So its good to first explore and know your data before jumping into the analysis phase.


In [ ]:
#Importing important libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [ ]:
#Bring in the train data
train_df=pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
#Columns in our data
train_df.columns

**So what now..?**

In order to get a better insight of our data is we need to get familiar with the features or columns pressent in our data.I know its a tedious job to go through each and every column but in ling run its really useful because it will tell use which features are going to affect our predections or are going to be useful for our target predictions.

**First we should analyze our sales price**

In [ ]:
train_df['SalePrice'].describe()

*So Here as we saw min sales price is 34900 so its obvisouly greater than zero.It kinda tells that our train data is realistic enough.*

In [ ]:
#plot of our sales price
sns.distplot(train_df['SalePrice']);

* **Okay so this tells our sales price is not noramlly distributes.**
* **And is kinda positevly skewed**

In [ ]:
#Lets check if we are right
print('skweness %f'% train_df['SalePrice'].skew())
print('Kurtness %f'% train_df['SalePrice'].kurt())

*OHkay so we were right as calculated skewness is nowhere close to 0 so hence data isnt normally disrtibuted or is not symmetrical and because kurtness is more than 3 so the data has a heavy peak.*

**SalePrice relationship with other features**

Now that we know a bit about our saleprice distributions lets study its relationships with other features in our data -set

In [ ]:
#Scatter plot between salprice and grlivearea
GrliveAr_Sp=pd.concat([train_df['GrLivArea'],train_df['SalePrice']],axis=1);
GrliveAr_Sp.plot.scatter(x='GrLivArea',y='SalePrice',ylim=(0,800000))

*wokayy!So SalePrice and GrlivArea kinda have a strong relationship(Linear Relationship) beween them*

> Time to check SalePrice VS TotalBsmtSF

In [ ]:
TotalBsmtSF_sp=pd.concat([train_df['TotalBsmtSF'],train_df['SalePrice']],axis=1)
TotalBsmtSF_sp.plot.scatter(x='TotalBsmtSF',y='SalePrice',ylim=(0,800000))

*Here also we can observe a strong relationship*
But wait there is a outlier in our data when TotalBsmtSF is 6000 .

**Relationship with categorical features**

In [ ]:
#box plot overallqual/saleprice
OverallQual_sp=pd.concat([train_df['OverallQual'],train_df['SalePrice']],axis=1)
plt.subplots(figsize=(8, 6))
sns.boxplot(x='OverallQual',y='SalePrice',data=OverallQual_sp)

Apart from some outliers saleprice goes pretty well with overallQall too..

In [ ]:
var = 'YearBuilt'
data = pd.concat([train_df['SalePrice'], train_df[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

**This plot between saleprice and year built dosent really shows a strong relationship between sales price but one thing we can undeerstand from this plot is that houses built in recent years have high prices**

**Lets improve our approach....!**

Till now e followed our intitution and just checked the relationship which we though were strong between saleprice but if we keep follwing this approach we will end up in a long tidious and messey exploration so let's try something.
something via which we can understand the realtionship of all the features with saleprice..
what is it then..? 
the answer is .

**A correlation heatmap**

In [ ]:
correlation=train_df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(correlation,vmax=0.9,square=True)

In [ ]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = correlation.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train_df[cols].values.T)
sns.set(font_scale=1.25)
f, ax = plt.subplots(figsize=(10, 8))
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()


In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train_df[cols], size = 2.5)
plt.show();

**Time to deal with missing data in our dataset**

In [ ]:
total_Null=train_df.isnull().sum().sort_values(ascending=False)
percent=(train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total_Null, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
#dealing with missing data
train_df = train_df.drop((missing_data[missing_data['Total'] > 1]).index,1)
train_df = train_df.drop(train_df.loc[train_df['Electrical'].isnull()].index)
train_df.isnull().sum().max() 
#just checking that there's no missing data missing anymore

In [ ]:
len(train_df.columns)

**Our liars**

*Outliers can markedly affect our models and can be a valuable source of information, providing us insights about specific behaviours.*
*Outliers is a complex subject and it deserves more attention. Here, we'll just do a quick analysis through the standard deviation of 'SalePrice' and a set of scatter plots.*

In [ ]:
#standardizing data
saleprice_scaled = StandardScaler().fit_transform(train_df['SalePrice'][:,np.newaxis]);
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

In [ ]:
train_df.sort_values(by='GrLivArea',ascending=False)[:2]

In [ ]:
train_df=train_df.drop(train_df[train_df['Id']==1298].index)
train_df=train_df.drop(train_df[train_df['Id']==524].index)


**Do we know salePrice ..?
**

The question might sound silly at first but we need to answe it .
We have to know who is salePrice 
What trends does it shows us

**Is SalePrice normal..?**

In [ ]:
sns.distplot(train_df['SalePrice'],fit=norm)

So We have answered our question 
Sale Price isnt normal for sure.

In [ ]:
sts=stats.probplot(train_df['SalePrice'],plot=plt)

Its clear again that it isnt normal because it dosent follow the diagnoal part

In [ ]:
#to deal with this issue we apply lograthmic transformation
train_df['SalePrice'] = np.log(train_df['SalePrice'])


In [ ]:
sns.distplot(train_df['SalePrice'],fit=norm)

In [ ]:
sts=stats.probplot(train_df['SalePrice'],plot=plt)

**Problem solved hurray!!**

In [ ]:
#histogram and normal probability plot
sns.distplot(train_df['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(train_df['GrLivArea'], plot=plt)


In [ ]:
#to deal with this issue we apply lograthmic transformation
train_df['GrLivArea'] = np.log(train_df['GrLivArea'])


In [ ]:
#histogram and normal probability plot
sns.distplot(train_df['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(train_df['GrLivArea'], plot=plt)


In [ ]:
#histogram and normal probability plot
sns.distplot(train_df['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(train_df['TotalBsmtSF'], plot=plt)


***As we can see there a number of houses with no basement and value zero dosent allow us to do log transformations.***

So what should we do to deal with it .?

In [ ]:
train_df['HasBasement']=pd.Series(len(train_df['TotalBsmtSF']),index=train_df.index)
train_df['HasBasement']=0

In [ ]:
train_df.loc[train_df['TotalBsmtSF']>0,'HasBasement']=1

In [ ]:
#transform data
train_df.loc[train_df['HasBasement']==1,'TotalBsmtSF'] = np.log(train_df['TotalBsmtSF'])

In [ ]:
sns.distplot(train_df[train_df['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(train_df[train_df['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)


In [ ]:
#scatter plot
plt.scatter(train_df['GrLivArea'], train_df['SalePrice']);


In [ ]:
train_df=pd.get_dummies(train_df)